In [78]:
#Preliminaries:
#
#Edges are defined as 2-tuples (x,y) where x,y are the nodes involved
#Wedges are defined as 3-tuples (x,y,z) where x,y,z are the nodes involved

In [153]:
#Imports
import networkx as nx
import math
import random
import sys

In [154]:
#Check if an edge completes a triangle
def check_wedge_closes(wedge,edge):
    return ((edge[0] == wedge[2] and edge[1] == wedge[0]) or (edge[1] == wedge[2] and edge[0] == wedge[0]))

In [155]:
#Update total number of wedges
def update_tot_wedges(edge_reservoir,wedge_reservoir,replaced_edges,input_edge,current_total_wedges):
    #sys.stdout.write(" ".join(str(x) for x in replaced_edges))
    #First we check how many of the replaced edges have duplicates in the edge reservoir
    for i in range(0,len(edge_reservoir)):
        index = -1
        for j in range(0,len(replaced_edges)):
            if (edge_reservoir[i][0] == replaced_edges[j][0] and edge_reservoir[i][1] == replaced_edges[j][1]):
                index = j
        if index != -1:
            del replaced_edges[index]
    #sys.stdout.write("#")
    #sys.stdout.write(" ".join(str(x) for x in replaced_edges))
    #sys.stdout.write("#")
    #The remaining edges did not have any duplicates. We check in how many wedges they participate
    invalids = 0
    checked_wedges = []
    if len(replaced_edges) > 0:
        for wedge in wedge_reservoir:
            if (wedge in checked_wedges): continue #If we counted wedge already move on
            for redge in replaced_edges:
                #We need to check all permutations
                if (wedge[0] == redge[0] and wedge[1] == redge[1]) or (wedge[0] == redge[1] and wedge[1] == redge[0]) or (wedge[1] == redge[0] and wedge[2] == redge[1]) or (wedge[1] == redge[1] and wedge[2] == redge[0]):
                    #This replaced edge participated in a wedge. This wedge is no longer valid
                    invalids += 1
                    checked_wedges.append(wedge)
    
    #print(invalids)
    
    #Finally, we count and store the new number of wedges that can be created from the new edge
    #We keep a list of "used edges" for the duplicates so no duplicate wedges are created
    new_wedges = []
    used_edges = []
    for edge in edge_reservoir:
        check = False
        for used_edge in used_edges:
            if (used_edge[0] == edge[0] and used_edge[1] == edge[1]):
                check = True
                break
        if(check):
            continue
        used = False
        if (edge[0] == input_edge[1] and edge[1] != input_edge[0]):
            new_wedges.append((input_edge[0],input_edge[1],edge[1]))
            used = True
        elif (edge[1] == input_edge[1] and edge[0] != input_edge[0]):
            new_wedges.append((input_edge[0],input_edge[1],edge[0]))
            used = True
        elif (edge[0] == input_edge[0] and edge[1] != input_edge[1]):
            new_wedges.append((input_edge[1],input_edge[0],edge[1]))
            used = True
        elif (edge[1] == input_edge[0] and edge[0] != input_edge[1]):
            new_wedges.append((input_edge[1],input_edge[0],edge[0]))
            used = True
        if used:
            used_edges.append(edge)
            
    #The overall change in the total wedges is:
    new_total_wedges = current_total_wedges - invalids + len(new_wedges)
    
    #We return this number as well as the list of new wedges
    return (new_total_wedges, new_wedges)
        
        

In [156]:
#Algorithm parameters
edge_reservoir_size = 1000
wedge_reservoir_size = 1000

In [157]:
#Initialization
edge_reservoir = [(-1,-1)] * edge_reservoir_size
wedge_reservoir = [(-1,-1,-1)] * wedge_reservoir_size
is_an_antiedge = [0] * edge_reservoir_size #Binary list marking anti-edges
is_closed = [False] * wedge_reservoir_size #Binary list marking closed wedges
#edges = open('C:\\amazon_wdels.txt','r')
edges = open('C:\\facebook_wdels.txt','r')
#edges = open('C:\\wtest.txt','r')
counter = 0
total_number_of_wedges = 0 #This is different than the length of the wedge reservoir!
results = []

#For every edge:
for input_edge_raw in edges:
    if(components[0] == '0'): continue #Skip deletions for now
    counter += 1 #Increase edge counter
    replaced_edges = [] #A list containing replaced edges
    components = input_edge_raw.split(' ')
    v1 = int(components[1])
    v2 = int(components[2])
    input_edge = (v1,v2)
    #Check if particular edge closes a wedge
    for i in range(0,len(wedge_reservoir)):
        if(check_wedge_closes(wedge_reservoir[i],input_edge)):
            is_closed[i] = True
    #Replace edges (reservoir sampling)
    for i in range(0,len(edge_reservoir)):
        die = random.random()
        if die < (1.0/counter):
            replaced_edges.append(edge_reservoir[i])
            edge_reservoir[i] = input_edge
            
    #Don't do stuff until reservoir is initialized
    if(counter==1): continue
    
    #if(counter==2):
    #print(replaced_edges)
    #break
    
    #If an update happened
    new_wedges = []
    tot_wedges = 0 #For initialization
    if len(replaced_edges) > 0:
        #Calculate new total number of wedges
        tmp = update_tot_wedges(edge_reservoir,wedge_reservoir,replaced_edges,input_edge,total_number_of_wedges)
        tot_wedges = tmp[0]
        new_wedges = tmp[1]
        #print(tmp)
        
        #Replace wedges in the wedge reservoir
        if(len(new_wedges)>0 and tot_wedges>0):
            for i in range(0,len(wedge_reservoir)):
                die = random.random()
                if die < (len(new_wedges)/float(tot_wedges)):
                    #Pick a uniform new wedge
                    random_wedge_idx = random.randint(0,len(new_wedges)-1)
                    wedge_reservoir[i] = new_wedges[random_wedge_idx]
                    is_closed[i] = False
            
    #Finally we can now approximate the number of triangles in the graph
    #This will change with the anti-edges
    closed = 0
    for wedge in is_closed:
        if wedge == True:
            closed += 1
    
    total_number_of_wedges = tot_wedges
    if(counter%1000 == 0):
        fraction_p = closed/len(is_closed)
        kt = 3*fraction_p
        triangles = ((fraction_p*(math.pow(counter,2)))/(edge_reservoir_size*(edge_reservoir_size-1)))*total_number_of_wedges
        #print(triangles)
        print(total_number_of_wedges)
    

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


KeyboardInterrupt: 

In [137]:
print(is_closed)
print(edge_reservoir)
print(wedge_reservoir)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
[(393830, 507828), (110323, 297774), (267443, 281305), (112938, 186548), (81997, 372871), (19621, 314051), (144703, 404383), (54826, 540068), (50495, 420147), (156441, 386722), (371043, 477015), (32148, 536259), (116840, 338053), (129702, 164162), (1380, 98710), (110038, 499315), (349398, 514782), 

In [65]:
check_wedge_closes((1,2,3),(1,1))

False

In [123]:
a = [(1,2,3),(4,5,6)]
print((1,2,3) in a)

True
